<a href="https://colab.research.google.com/github/monda00/horse-race-notebook/blob/master/make_import_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# アプリにインポートするデータ形式に変換する

# ライブラリ・データ読み込み

In [1]:
import numpy as np
import pandas as pd
import re
import collections
from tqdm import tqdm
import datetime

In [2]:
DATA_PATH = '/content/drive/My Drive/data/horse-race/'

In [3]:
race_df = pd.read_csv(DATA_PATH + 'race_ex.csv')
horse_df = pd.read_csv(DATA_PATH + 'horse_ex.csv')

# 前処理

必要なデータ

## ７頭以下のレースを削除

In [4]:
horse_df['race_id'].value_counts()

202008030303    18
201904021003    18
201910020707    18
201907010209    18
201908030105    18
                ..
201945051701     4
201946061610     4
202035052403     4
201947070506     3
201935071404     3
Name: race_id, Length: 24221, dtype: int64

In [5]:
drop_races_s = horse_df['race_id'].value_counts() <= 7
drop_race_id = drop_races_s[drop_races_s == True].index

for race_id in drop_race_id:
  race_df = race_df[race_df['race_id'] != race_id]

for race_id in drop_race_id:
  horse_df = horse_df[horse_df['race_id'] != race_id]

In [6]:
horse_df['race_id'].value_counts()

201910010805    18
201910010107    18
201906050612    18
201908030105    18
201904030106    18
                ..
201936082609     8
201954021002     8
201950070403     8
201943080912     8
202047061904     8
Name: race_id, Length: 22630, dtype: int64

## 性別と年齢の分割

In [7]:
gen = []
age = []
for i in range(len(horse_df)):
  age_v = horse_df.iloc[i]['age']
  gen.append(re.search(r'(.)(\d{1,2})', age_v).group(1))
  age.append(re.search(r'(.)(\d{1,2})', age_v).group(2))

In [8]:
horse_df['age'] = age
horse_df['gen'] = gen

## 負担重量と体重と増減

In [9]:
horse_df['burden_weight'] = horse_df['weight']

In [10]:
weight = []
weight_diff = []
for i in range(len(horse_df)):
  horse_weight = horse_df.iloc[i]['horse_weight']
  if horse_weight == '計不':
    weight.append('計不')
    weight_diff.append('計不')
  else:
    weight.append(int(re.search(r'(.*)(\(.*?\))', horse_weight).group(1)))
    weight_diff.append(re.search(r'(.*)(\(.*?\))', horse_weight).group(2).replace('(', '').replace(')', ''))

In [11]:
horse_df['weight'] = weight
horse_df['weight_diff'] = weight_diff

## 日時

In [12]:
race_df

,Unnamed: 0,clockwise,date,distance,field_condition,field_type,name,place,race_id,race_round,start_time,weather
0,0,右,2020/6/22,1400,不,ダ,タイタン賞競走,大井,202044062212,12R,20:50,雨
1,1,右,2020/6/22,1800,不,ダ,ポートサイド賞競走,大井,202044062211,11R,20:10,雨
2,2,右,2020/6/22,1200,不,ダ,C1九 十11,大井,202044062210,10R,19:30,雨
3,3,右,2020/6/22,1400,不,ダ,C1九 十11,大井,202044062209,9R,18:55,雨
4,4,右,2020/6/22,1600,不,ダ,C1九 十11,大井,202044062208,8R,18:20,雨
...,...,...,...,...,...,...,...,...,...,...,...,...
24215,27514,左,2019/1/1,1500,良,ダ,3歳4,川崎,201945010106,6R,13:50,晴
24216,27515,左,2019/1/1,900,良,ダ,タンチョウ賞,川崎,201945010105,5R,13:20,晴
24217,27516,左,2019/1/1,1400,良,ダ,3歳5,川崎,201945010104,4R,12:50,晴
24218,27517,左,2019/1/1,1400,良,ダ,C3七 八,川崎,201945010103,3R,12:20,晴


In [13]:
race_df['date_time'] = pd.to_datetime(race_df['date'] + ' ' + race_df['start_time'])

## 出走馬数

In [14]:
race_df['race_horse_number'] = list(collections.Counter(list(horse_df['race_id'].values)).values())

## 計測不能の置換

In [15]:
horse_df = horse_df.replace('計不', 0)

## タイムの変換

In [16]:
for i in tqdm(range(len(horse_df))):
  dt = datetime.datetime.strptime(horse_df.iloc[i]['time'], '%M:%S.%f')
  horse_df.loc[i, 'time'] = datetime.timedelta(minutes=dt.minute, seconds=dt.second, milliseconds=dt.microsecond).total_seconds()

100%|██████████| 253636/253636 [25:41<00:00, 164.55it/s]


## 過去レースのデータ

In [17]:
race_df['date'] = pd.to_datetime(race_df['date'])
horse_df['race_date'] = pd.to_datetime(horse_df['race_date'])
race_df = race_df.sort_values(by=['date', 'race_id']).reset_index()
horse_df = horse_df.sort_values(by=['race_date', 'race_id', 'rank']).reset_index()

In [18]:
past_race_columns_base = ['odd', 'popular', 'rank', 'time', 'elapsed_day', 'distance', 'field_type', 'field_condition', 'weather']
past_race_num = ['one', 'two', 'three', 'four', 'five']
past_race_columns = []
for n in past_race_num:
  for c in past_race_columns_base:
    past_race_columns.append('{}_before_{}'.format(n, c))

In [19]:
horse_df[past_race_columns] = 0

In [20]:
for i in tqdm(range(243106, len(horse_df))):
  horse_name = horse_df.iloc[i]['name']
  race_date = horse_df.iloc[i]['race_date']
  past_num = 0

  df = horse_df.iloc[:i].query('name == "{}"'.format(horse_name))
  for j in range(len(df)-1, -1, -1):
    race_date_before = df.iloc[j]['race_date']
    race = race_df[race_df['race_id'] == df.iloc[j]['race_id']]
    horse_df.loc[i, '{}_before_{}'.format(past_race_num[past_num], 'odd')] = df.iloc[j]['win']
    horse_df.loc[i, '{}_before_{}'.format(past_race_num[past_num], 'popular')] = df.iloc[j]['popular']
    horse_df.loc[i, '{}_before_{}'.format(past_race_num[past_num], 'rank')] = df.iloc[j]['rank']
    horse_df.loc[i, '{}_before_{}'.format(past_race_num[past_num], 'time')] = df.iloc[j]['time']
    horse_df.loc[i, '{}_before_{}'.format(past_race_num[past_num], 'elapsed_day')] = abs(race_date - race_date_before).days
    horse_df.loc[i, '{}_before_{}'.format(past_race_num[past_num], 'distance')] = race.distance.values
    horse_df.loc[i, '{}_before_{}'.format(past_race_num[past_num], 'field_type')] = race.field_type.values
    horse_df.loc[i, '{}_before_{}'.format(past_race_num[past_num], 'field_condition')] = race.field_condition.values
    horse_df.loc[i, '{}_before_{}'.format(past_race_num[past_num], 'weather')] = race.weather.values
    past_num += 1
    
    if past_num >= 5:
      break

100%|██████████| 20562/20562 [29:23<00:00, 11.66it/s]


## データ数を制限

In [21]:
race_df['date'] = pd.to_datetime(race_df['date'])
horse_df['race_date'] = pd.to_datetime(horse_df['race_date'])

In [22]:
race_df = race_df[race_df['date'] > '2020-05-31']
horse_df = horse_df[horse_df['race_date'] > '2020-05-31']

# 確認

In [23]:
horse_df

,index,Unnamed: 0,agari,age,frame_number,horse_number,horse_weight,jockey,name,popular,race_date,race_id,race_name,rank,time,weight,win,gen,burden_weight,weight_diff,one_before_odd,one_before_popular,one_before_rank,one_before_time,one_before_elapsed_day,one_before_distance,one_before_field_type,one_before_field_condition,one_before_weather,two_before_odd,two_before_popular,two_before_rank,two_before_time,two_before_elapsed_day,two_before_distance,two_before_field_type,two_before_field_condition,two_before_weather,three_before_odd,three_before_popular,three_before_rank,three_before_time,three_before_elapsed_day,three_before_distance,three_before_field_type,three_before_field_condition,three_before_weather,four_before_odd,four_before_popular,four_before_rank,four_before_time,four_before_elapsed_day,four_before_distance,four_before_field_type,four_before_field_condition,four_before_weather,five_before_odd,five_before_popular,five_before_rank,five_before_time,five_before_elapsed_day,five_before_distance,five_before_field_type,five_before_field_condition,five_before_weather
243106,10876,11964.0,37.4,6,6.0,6.0,487(+2),山本政聡,バトルクウ,1.0,2020-06-01,2.020351e+11,C2,1.0,931,487.0,2.0,牝,54.0,+2,5.5,4.0,5.0,1013,13,1400,ダ,不,雨,7.9,4.0,5.0,488,28,1400,ダ,良,晴,3.8,2.0,4.0,782,41,850,ダ,不,曇,4.0,2.0,3.0,485,48,1400,ダ,良,晴,22.1,7.0,8.0,90,72,850,ダ,良,曇
243107,10877,11965.0,37.6,4,4.0,4.0,432(+1),菅原辰徳,リュウグウオー,2.0,2020-06-01,2.020351e+11,C2,2.0,156,432.0,4.9,牡,56.0,+1,13.3,4.0,6.0,605,7,1000,ダ,稍,晴,36.7,6.0,4.0,984,27,850,ダ,良,晴,43.9,7.0,3.0,613,48,850,ダ,良,晴,36.9,6.0,4.0,788,55,850,ダ,重,曇,23.7,4.0,9.0,491,64,1400,ダ,稍,曇
243108,10884,11972.0,37.1,5,8.0,8.0,436(+5),阿部英俊,コスモミクニグロ,4.0,2020-06-01,2.020351e+11,C2,3.0,900,436.0,6.0,牡,56.0,+5,11.5,5.0,5.0,120,6,1400,ダ,良,晴,4.8,3.0,2.0,92,21,1400,ダ,良,晴,11.7,5.0,5.0,712,28,1400,ダ,良,晴,119.2,10.0,5.0,807,55,1400,ダ,良,晴,18.4,5.0,5.0,218,68,1400,ダ,良,晴
243109,10885,11973.0,37.6,4,5.0,5.0,430(-7),高松亮,ストーリーズ,6.0,2020-06-01,2.020351e+11,C2,4.0,900,430.0,16.8,牝,54.0,-7,11.9,6.0,5.0,335,6,1400,ダ,良,晴,29.5,7.0,8.0,591,55,1400,ダ,重,曇,18.1,7.0,10.0,497,65,1400,ダ,稍,曇,3.4,2.0,10.0,70,154,1400,ダ,不,曇,2.3,1.0,4.0,623,343,1400,ダ,不,雨
243110,10886,11974.0,37.2,5,1.0,1.0,485(+12),岩本怜,アルファルミナス,7.0,2020-06-01,2.020351e+11,C2,5.0,323,485.0,20.4,牝,54.0,+12,4.1,2.0,8.0,516,7,1400,ダ,良,晴,9.2,6.0,5.0,685,14,1400,ダ,良,晴,80.0,9.0,7.0,281,35,1200,ダ,稍,雨,29.9,7.0,10.0,804,56,1200,ダ,良,晴,75.0,11.0,12.0,1045,182,1200,ダ,不,雨
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253631,41,41.0,39.2,7,7.0,12.0,551(+13),矢野貴之,フィールブリーズ,7.0,2020-06-22,2.020441e+11,タイタン賞競走,11.0,1005,551.0,22.7,牡,56.0,+13,3.7,2.0,5.0,989,77,1400,ダ,良,晴,8.6,4.0,9.0,975,98,1200,ダ,重,晴,3.4,1.0,1.0,906,115,1400,ダ,良,晴,5.5,3.0,1.0,459,137,1200,ダ,良,晴,4.4,3.0,1.0,620,150,1400,ダ,重,曇
253632,42,42.0,39.3,7,2.0,2.0,466(-5),町田直希,キングオブアームズ,13.0,2020-06-22,2.020441e+11,タイタン賞競走,12.0,306,466.0,97.2,セ,56.0,-5,95.5,11.0,12.0,218,19,1400,ダ,良,曇,6.6,4.0,4.0,593,138,1400,ダ,良,晴,105.8,10.0,5.0,1037,178,1400,ダ,重,晴,176.3,12.0,12.0,49,202,1800,ダ,不,晴,43.6,9.0,14.0,820,251,1600,ダ,重,曇
253633,83,83.0,39.2,7,1.0,1.0,523(+7),千田洋,プライズコレクター,11.0,2020-06-22,2.020441e+11,タイタン賞競走,13.0,891,523.0,58.4,牡,56.0,+7,12.2,5.0,4.0,1:44.4,538,1600,ダ,良,晴,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0
253634,84,84.0,39.8,4,4.0,6.0,477(+2),本田正重,レクスミノル,10.0,2020-06-22,2.020441e+11,タイタン賞競走,14.0,891,477.0,42.7,牡,56.0,+2,61.4,7.0,7.0,89,17,1600,ダ,良,晴,39.4,9.0,13.0,307,32,1800,ダ,不,曇,55.1,12.0,15.0,615,72,1700,ダ,良,晴,114.1,11.0,12.0,714,161,1800,ダ,稍,晴,29.7,9.0,13.0,880,177,1800,ダ,稍,晴


In [24]:
race_df

,index,Unnamed: 0,clockwise,date,distance,field_condition,field_type,name,place,race_id,race_round,start_time,weather,date_time,race_horse_number
21688,999,1108,左,2020-06-01,1000,良,ダ,C2,盛岡,202035060101,1R,12:10,晴,2020-06-01 12:10:00,8
21689,998,1107,左,2020-06-01,1000,良,ダ,C2,盛岡,202035060102,2R,12:40,晴,2020-06-01 12:40:00,9
21690,997,1106,左,2020-06-01,1200,良,ダ,C2九組,盛岡,202035060103,3R,13:10,晴,2020-06-01 13:10:00,9
21691,996,1105,左,2020-06-01,1200,良,ダ,C2八組,盛岡,202035060104,4R,13:45,晴,2020-06-01 13:45:00,9
21692,995,1104,左,2020-06-01,1200,良,ダ,C2七組,盛岡,202035060105,5R,14:20,晴,2020-06-01 14:20:00,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22625,4,4,右,2020-06-22,1600,不,ダ,C1九 十11,大井,202044062208,8R,18:20,雨,2020-06-22 18:20:00,12
22626,3,3,右,2020-06-22,1400,不,ダ,C1九 十11,大井,202044062209,9R,18:55,雨,2020-06-22 18:55:00,14
22627,2,2,右,2020-06-22,1200,不,ダ,C1九 十11,大井,202044062210,10R,19:30,雨,2020-06-22 19:30:00,14
22628,1,1,右,2020-06-22,1800,不,ダ,ポートサイド賞競走,大井,202044062211,11R,20:10,雨,2020-06-22 20:10:00,15


# エクスポート

In [25]:
use_columns_horse = [
                    'race_id',
                    'horse_number',
                    'frame_number',
                    'age',
                    'gen',
                    'weight',
                    'weight_diff',
                    'burden_weight'
]

use_columns_race = [
                    'race_id',
                    'name',
                    'place',
                    'race_horse_number',
                    'distance',
                    'clockwise',
                    'field_type',
                    'field_condition',
                    'weather',
                    'date_time'
]

In [26]:
use_columns_horse += past_race_columns

In [27]:
horse_df[use_columns_horse].to_csv(DATA_PATH+'import_horse.csv', index=False)
race_df[use_columns_race].to_csv(DATA_PATH+'import_race.csv', index=False)

In [28]:
horse_df[use_columns_horse]

,race_id,horse_number,frame_number,age,gen,weight,weight_diff,burden_weight,one_before_odd,one_before_popular,one_before_rank,one_before_time,one_before_elapsed_day,one_before_distance,one_before_field_type,one_before_field_condition,one_before_weather,two_before_odd,two_before_popular,two_before_rank,two_before_time,two_before_elapsed_day,two_before_distance,two_before_field_type,two_before_field_condition,two_before_weather,three_before_odd,three_before_popular,three_before_rank,three_before_time,three_before_elapsed_day,three_before_distance,three_before_field_type,three_before_field_condition,three_before_weather,four_before_odd,four_before_popular,four_before_rank,four_before_time,four_before_elapsed_day,four_before_distance,four_before_field_type,four_before_field_condition,four_before_weather,five_before_odd,five_before_popular,five_before_rank,five_before_time,five_before_elapsed_day,five_before_distance,five_before_field_type,five_before_field_condition,five_before_weather
243106,2.020351e+11,6.0,6.0,6,牝,487.0,+2,54.0,5.5,4.0,5.0,1013,13,1400,ダ,不,雨,7.9,4.0,5.0,488,28,1400,ダ,良,晴,3.8,2.0,4.0,782,41,850,ダ,不,曇,4.0,2.0,3.0,485,48,1400,ダ,良,晴,22.1,7.0,8.0,90,72,850,ダ,良,曇
243107,2.020351e+11,4.0,4.0,4,牡,432.0,+1,56.0,13.3,4.0,6.0,605,7,1000,ダ,稍,晴,36.7,6.0,4.0,984,27,850,ダ,良,晴,43.9,7.0,3.0,613,48,850,ダ,良,晴,36.9,6.0,4.0,788,55,850,ダ,重,曇,23.7,4.0,9.0,491,64,1400,ダ,稍,曇
243108,2.020351e+11,8.0,8.0,5,牡,436.0,+5,56.0,11.5,5.0,5.0,120,6,1400,ダ,良,晴,4.8,3.0,2.0,92,21,1400,ダ,良,晴,11.7,5.0,5.0,712,28,1400,ダ,良,晴,119.2,10.0,5.0,807,55,1400,ダ,良,晴,18.4,5.0,5.0,218,68,1400,ダ,良,晴
243109,2.020351e+11,5.0,5.0,4,牝,430.0,-7,54.0,11.9,6.0,5.0,335,6,1400,ダ,良,晴,29.5,7.0,8.0,591,55,1400,ダ,重,曇,18.1,7.0,10.0,497,65,1400,ダ,稍,曇,3.4,2.0,10.0,70,154,1400,ダ,不,曇,2.3,1.0,4.0,623,343,1400,ダ,不,雨
243110,2.020351e+11,1.0,1.0,5,牝,485.0,+12,54.0,4.1,2.0,8.0,516,7,1400,ダ,良,晴,9.2,6.0,5.0,685,14,1400,ダ,良,晴,80.0,9.0,7.0,281,35,1200,ダ,稍,雨,29.9,7.0,10.0,804,56,1200,ダ,良,晴,75.0,11.0,12.0,1045,182,1200,ダ,不,雨
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253631,2.020441e+11,12.0,7.0,7,牡,551.0,+13,56.0,3.7,2.0,5.0,989,77,1400,ダ,良,晴,8.6,4.0,9.0,975,98,1200,ダ,重,晴,3.4,1.0,1.0,906,115,1400,ダ,良,晴,5.5,3.0,1.0,459,137,1200,ダ,良,晴,4.4,3.0,1.0,620,150,1400,ダ,重,曇
253632,2.020441e+11,2.0,2.0,7,セ,466.0,-5,56.0,95.5,11.0,12.0,218,19,1400,ダ,良,曇,6.6,4.0,4.0,593,138,1400,ダ,良,晴,105.8,10.0,5.0,1037,178,1400,ダ,重,晴,176.3,12.0,12.0,49,202,1800,ダ,不,晴,43.6,9.0,14.0,820,251,1600,ダ,重,曇
253633,2.020441e+11,1.0,1.0,7,牡,523.0,+7,56.0,12.2,5.0,4.0,1:44.4,538,1600,ダ,良,晴,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0
253634,2.020441e+11,6.0,4.0,4,牡,477.0,+2,56.0,61.4,7.0,7.0,89,17,1600,ダ,良,晴,39.4,9.0,13.0,307,32,1800,ダ,不,曇,55.1,12.0,15.0,615,72,1700,ダ,良,晴,114.1,11.0,12.0,714,161,1800,ダ,稍,晴,29.7,9.0,13.0,880,177,1800,ダ,稍,晴


In [29]:
race_df[use_columns_race]

,race_id,name,place,race_horse_number,distance,clockwise,field_type,field_condition,weather,date_time
21688,202035060101,C2,盛岡,8,1000,左,ダ,良,晴,2020-06-01 12:10:00
21689,202035060102,C2,盛岡,9,1000,左,ダ,良,晴,2020-06-01 12:40:00
21690,202035060103,C2九組,盛岡,9,1200,左,ダ,良,晴,2020-06-01 13:10:00
21691,202035060104,C2八組,盛岡,9,1200,左,ダ,良,晴,2020-06-01 13:45:00
21692,202035060105,C2七組,盛岡,10,1200,左,ダ,良,晴,2020-06-01 14:20:00
...,...,...,...,...,...,...,...,...,...,...
22625,202044062208,C1九 十11,大井,12,1600,右,ダ,不,雨,2020-06-22 18:20:00
22626,202044062209,C1九 十11,大井,14,1400,右,ダ,不,雨,2020-06-22 18:55:00
22627,202044062210,C1九 十11,大井,14,1200,右,ダ,不,雨,2020-06-22 19:30:00
22628,202044062211,ポートサイド賞競走,大井,15,1800,右,ダ,不,雨,2020-06-22 20:10:00
